In [1]:
# 导入包
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import zipfile
import pickle
import pandas as pd
import numpy as np
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import paddle.optimizer as optim
import paddle.optimizer.lr as lr
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm
from paddle.io import DataLoader, TensorDataset

# 固定随机种子保证结果可复现
seed = 42
np.random.seed(seed)
paddle.seed(seed)

# 导入自定义包
sys.path.append("work")
from candle2 import Canva

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:686: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


# Loss函数

In [ ]:
import paddle
import paddle.nn as nn

class MyMSE(nn.Layer):
    def __init__(self):
        super(MyMSE, self).__init__()
    
    def forward(self, input, target):
        # 获取 target 的最后一个非 nan 值
        target_last = paddle.where(paddle.isnan(target), paddle.to_tensor(float('nan')), target).min(axis=1, keepdim=True)
        
        # 计算均方误差（MSE）
        mse = paddle.mean((input - target_last) ** 2)
        
        return mse

# 模型-Multichannel

In [ ]:
class DilatedCNN(nn.Layer):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super(DilatedCNN, self).__init__()
        self.conv = nn.Conv1D(
            in_channels, 
            out_channels, 
            kernel_size, 
            padding=(kernel_size - 1) // 2 * dilation, 
            dilation=dilation
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

class BiLSTM(nn.Layer):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim, 
            hidden_dim, 
            num_layers, 
            direction='bidirectional', 
            time_major=False
        )

    def forward(self, x):
        x, _ = self.lstm(x)
        return x


class Attention(nn.Layer):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.softmax = nn.Softmax(axis=1)
        self.context_vector = paddle.create_parameter(
            shape=[hidden_dim * 2], 
            dtype='float32', 
            default_initializer=paddle.nn.initializer.Normal()
        )

    def forward(self, lstm_out):
        attn_weights = nn.ReLU()(self.attn(lstm_out))
        attn_weights = paddle.matmul(attn_weights, self.context_vector)
        attn_weights = self.softmax(attn_weights)
        context = paddle.matmul(attn_weights.unsqueeze(1), lstm_out).squeeze(1)
        return context

class multichannel(nn.Layer):
    def __init__(self, input_dim, cnn_out_channels=128, cnn_kernel_size=3, lstm_hidden_dim=256, lstm_num_layers=2, output_dim=1, dropout=0.1):
        super(multichannel, self).__init__()
        self.embedding = nn.Linear(input_dim**2*3, 128)
        paddle.nn.initializer.KaimingUniform()(self.embedding.weight)
        paddle.nn.initializer.Constant(value=0.0)(self.embedding.bias)
        self.dilated_cnn1 = DilatedCNN(128, cnn_out_channels, cnn_kernel_size, dilation=1)
        self.dilated_cnn2 = DilatedCNN(128, cnn_out_channels, cnn_kernel_size, dilation=2)
        self.dilated_cnn3 = DilatedCNN(128, cnn_out_channels, cnn_kernel_size, dilation=3)

        self.lstm_cnn = BiLSTM(cnn_out_channels * 3, lstm_hidden_dim, lstm_num_layers)
        self.lstm_direct = BiLSTM(128, lstm_hidden_dim, lstm_num_layers)
        for layer in [self.lstm_cnn, self.lstm_direct]:
            for param in layer.parameters():
                if param.ndim >= 2:
                    paddle.nn.initializer.XavierUniform()(param)
                else:
                    paddle.nn.initializer.Constant(value=0.0)(param)
                    
        self.local_attention = Attention(lstm_hidden_dim)
        self.global_attention = Attention(lstm_hidden_dim * 2)
        self.dropout = nn.Dropout(dropout)
        self.q = nn.Linear(lstm_hidden_dim * 4, lstm_hidden_dim * 2)
        self.fc = nn.Linear(lstm_hidden_dim * 2, output_dim)
        paddle.nn.initializer.XavierUniform()(self.q.weight)
        paddle.nn.initializer.XavierUniform()(self.fc.weight)
        paddle.nn.initializer.Constant(value=0.0)(self.q.bias)
        paddle.nn.initializer.Constant(value=0.0)(self.fc.bias)

    def forward(self, x):
        x = (x - 128.0) / 255.0
        x = x.reshape((x.shape[0], -1))
        x = self.embedding(x).unsqueeze(1)
        x = paddle.transpose(x, [0, 2, 1]) 
        # # print("Embedding Output Shape:", x.shape)
        # if x.shape[1:] != [128, 128]:
        #     pad_shape = [0, 128 - x.shape[2], 0, 128 - x.shape[1]]
        #     x = F.pad(x, pad_shape, mode='constant', value=0)  # 扩充到 [batch_size, 128, 128]
        # 多通道膨胀卷积
        x1 = self.dilated_cnn1(x)
        x2 = self.dilated_cnn2(x)
        x3 = self.dilated_cnn3(x)
        x_cnn = paddle.concat((x1, x2, x3), axis=1)
        x_cnn = paddle.transpose(x_cnn, [0, 2, 1])  # (batch_size, seq_len, cnn_out_channels * 3)
        # print("CNN Output Shape:", x_cnn.shape)

        lstm_cnn_out = self.lstm_cnn(x_cnn)
        local_cnn_attn_out = self.local_attention(lstm_cnn_out)

        lstm_direct_out = self.lstm_direct(paddle.transpose(x, [0, 2, 1]))
        local_direct_attn_out = self.local_attention(lstm_direct_out)


        combined_local_attn_out = paddle.concat((local_cnn_attn_out, local_direct_attn_out), axis=1)
        
        global_attn_out = self.global_attention(paddle.unsqueeze(combined_local_attn_out, axis=1)).squeeze(1)
        

        output = self.dropout(global_attn_out)
        # print(output.shape)
        output = self.q(output)
        output = self.fc(output)
        return output

# Loader创建

In [ ]:
# feature1~3  对应train_indices1.json
# feature4~6  对应2
# feature7~8  对应3

# 因为受限与显存，这里仅使用0-5w数据作为训练， 5w-7w数据作为测试

In [ ]:
# 转换为 Paddle 张量
X_train = np.concatenate([np.load(f"work/train_dataset/train_features_{i}.npy") for i in range(1, 2)])[:50000]
y_train = np.load("work/train_dataset/train_labels1.npy")[:50000]

X_train = paddle.to_tensor(X_train, dtype=paddle.float32).unsqueeze(1)
y_train = paddle.to_tensor(y_train, dtype=paddle.float32)

# 创建数据加载器
train_dataset = TensorDataset([X_train, y_train])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
# 取 500000 条之后的作为验证集
X_val = np.concatenate([np.load(f"work/train_dataset/train_features_{i}.npy") for i in range(1, 2)])[50000:70000]
y_val = np.load("work/train_dataset/train_labels3.npy")[50000:70000]

X_val = paddle.to_tensor(X_val, dtype=paddle.float32).unsqueeze(1)
y_val = paddle.to_tensor(y_val, dtype=paddle.float32)

# 创建数据加载器
test_dataset = TensorDataset([X_val, y_val])
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Baseline + 梯度剪裁

In [ ]:
# 模型创建
model = multichannel(input_dim=160)

In [ ]:
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import paddle.optimizer as optim
import paddle.optimizer.lr as lr
from copy import deepcopy
from paddle.amp import GradScaler, auto_cast
import pandas as pd

result_dict = {
    "iter": None,
    "correlation": None,
    "net": None,
}
    
# 模型路径
save_path = 'work/trained_model/multichannel_gradclip_newdataset.pdparams'

# 定义损失函数
criterion = MyMSE()

# 定义学习率调度器
scheduler = lr.StepDecay(learning_rate=5e-5, step_size=50, gamma=0.01)

# # 初始化优化器并应用学习率调度器
# optimizer = optim.Adam(learning_rate=scheduler, parameters=model.parameters())

# 初始化最佳相关系数和最佳模型路径
result_dict["correlation"] = -np.inf  # 初始化为负无穷


# 定义梯度剪裁策略（例如：根据梯度范数剪裁，设置阈值为1.0）
grad_clip = paddle.nn.ClipGradByNorm(clip_norm=1.0)  # 也可以使用 ClipGradByValue(clip_value=0.5) 根据需要调整阈值

# 初始化优化器并应用学习率调度器和梯度剪裁
optimizer = paddle.optimizer.Adam(
    learning_rate=scheduler, 
    parameters=model.parameters(), 
    grad_clip=grad_clip  # 应用梯度剪裁
)

num_epochs = 20  # 可以更改
scaler = GradScaler()
for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.clear_grad()  # 清空梯度

        # 检查 NaN 或 Infinity 的输入和目标
        if np.any(np.isnan(inputs.numpy())) or np.any(np.isinf(inputs.numpy())):
            print("Inputs contain NaN or Infinity.")
            continue  # 跳过这个批次
        if np.any(np.isnan(targets.numpy())) or np.any(np.isinf(targets.numpy())):
            print("Targets contain NaN or Infinity.")
            continue  # 跳过这个批次

        # 混合精度训练
        with paddle.amp.auto_cast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        # 检查 loss 是否为 NaN
        if np.isnan(loss.numpy()):
            print("Loss is NaN. Skipping this batch.")
            continue  # 跳过这个批次

        # 使用 GradScaler 缩放损失并反向传播
        scaled_loss = scaler.scale(loss)  # 缩放损失
        scaled_loss.backward()  # 反向传播
        scaler.minimize(optimizer, scaled_loss)  # 优化器更新并取消缩放

        optimizer.clear_grad()  # 清除梯度

        if batch_idx % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}], Loss: {loss.numpy():.4f}')
    
    # 更新学习率和 scaler
    scheduler.step()
    scaler.update()

    # 每轮训练结束后可以检查或更新 scaler 的缩放值
    scaler.update()
    
    # 验证模型并计算相关系数
    model.eval()  # 设置模型为评估模式
    y_val_pred = []
    with paddle.no_grad():
        total_loss = 0.0
        for inputs, targets in test_loader:
            outputs = model(inputs)
            y_val_pred.append(outputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
        print(f'Average Test Loss: {total_loss / len(test_loader):.4f}')
    
    y_val_pred_arr = paddle.concat(y_val_pred, axis=0)
    y_val_np = y_val.numpy().reshape(1, -1)
    y_val_pred_np = y_val_pred_arr.numpy().reshape(1, -1)

    # 这里的 repeat 是为了让 y_val_pred 也能一周有 5 个值，与真实值对应
    y_val_pred_np = np.repeat(y_val_pred_np, 5, axis=1)

    # 计算预测值与真实值的相关系数
    correlation = pd.Series(y_val_pred_np.reshape(-1, )).corr(pd.Series(y_val_np.reshape(-1, )))
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Correlation: {correlation:.4f}')
    
    # 保存最佳模型
    if correlation > result_dict["correlation"]:
        result_dict["iter"] = epoch
        result_dict["correlation"] = correlation
        result_dict["net"] = deepcopy(model.state_dict())
        
paddle.save(result_dict, save_path)
print(f'Best model saved with correlation: {result_dict["correlation"]:.4f}, at iter {result_dict["iter"]}')

# 模型验证

In [ ]:
model = multichannel(input_dim=160)

# 加载训练好的模型状态字典。这里指定加载的是最后 1 个 epoch 的模型参数
model_state_dict = paddle.load(f'work/trained_model/multichannel_gradclip_newdataset.pdparams')

# 将状态字典加载到模型中
model.set_state_dict(model_state_dict['net'])

In [ ]:
# 评估模型
y_val_pred = []
criterion = MyMSE()
model.eval()
with paddle.no_grad():
    total_loss = 0.0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        y_val_pred.append(outputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
    print(f'Average Test Loss: {total_loss / len(test_loader):.4f}')


y_val_pred_arr = paddle.concat(y_val_pred, axis=0)
y_val_np = y_val.numpy().reshape(1, -1)
y_val_pred_np = y_val_pred_arr.numpy().reshape(1, -1)

# 这里的 repeat 是为了让 y_val_pred 也能一周有 5 个值，与真实值对应
y_val_pred_np = np.repeat(y_val_pred_np, 5, axis=1)

# 计算预测值与真实值的相关系数
pd.Series(y_val_pred_np.reshape(-1, )).corr(pd.Series(y_val_np.reshape(-1, )))